In [4]:
import os

import pandas as pd

from dataset.creating_dataset import min_max_scale_data
from dataset.splitting_dataset import split_dataframe
from training.train_and_evaluate_models import train_and_evaluate_models


PATH_TO_DATASET = os.path.join(
    "..", "..", "preprocessed_bestiaries", "bestiaries_full.csv"
)
TEST_RESULT_FILE = os.path.join("results", "results_test_scenarios_probit.xlsx")
TRAIN_RESULT_FILE = os.path.join("results", "results_train_scenarios_probit.xlsx")


bestiaries = pd.read_csv(PATH_TO_DATASET, index_col=0)
bestiaries = min_max_scale_data(bestiaries)

X_train, X_test, y_train, y_test = split_dataframe(bestiaries)

y_train += 1
y_test += 1

In [3]:
X_test.head()

,str,dex,con,int,wis,cha,ac,hp
428,0.529412,0.588235,0.428571,0.600000,0.470588,0.529412,0.358491,0.131886
429,0.529412,0.411765,0.285714,0.400000,0.352941,0.470588,0.320755,0.048414
430,0.470588,0.352941,0.428571,0.200000,0.294118,0.352941,0.283019,0.035058
431,0.470588,0.529412,0.357143,0.333333,0.411765,0.294118,0.490566,0.257095
432,0.470588,0.352941,0.142857,0.333333,0.529412,0.352941,0.339623,0.081803


In [11]:
import torch


# Hyperparameters
random_seed = 1
learning_rate = 0.05
num_epochs = 100
batch_size = 128

# Architecture
NUM_CLASSES = 23

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Training on", DEVICE)

Training on cpu


In [6]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, feature_array, label_array, dtype=np.float32):

        self.features = feature_array.astype(np.float32)
        self.labels = label_array

    def __getitem__(self, index):
        inputs = self.features[index]
        label = self.labels[index]
        return inputs, label

    def __len__(self):
        return self.labels.shape[0]

In [7]:
import torch
from torch.utils.data import DataLoader


# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = MyDataset(X_train.to_numpy(), y_train.to_numpy())
test_dataset = MyDataset(X_test.to_numpy(), y_test.to_numpy())


train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,  # want to shuffle the dataset
    num_workers=0,
)  # number processes/CPUs to use


# Checking the dataset
for inputs, labels in train_loader:
    print("Input batch dimensions:", inputs.shape)
    print("Input label dimensions:", labels.shape)
    break

Input batch dimensions: torch.Size([128, 50])
Input label dimensions: torch.Size([128])


In [8]:
from torch import sigmoid
from torch import nn
from coral_pytorch.layers import CoralLayer


class MLP(nn.Module):
    def __init__(self, input_size: int, num_classes: int):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
        )
        self.fc = CoralLayer(size_in=50, num_classes=num_classes)

    def forward(self, x):
        x = self.network(x)

        ##### Use CORAL layer #####
        logits = self.fc(x)
        probas = torch.sigmoid(logits)
        ###--------------------------------------------------------------------###

        return logits, probas

    def predict_proba(self, x):
        return sigmoid(self(x))

    def predict(self, x, threshold: float = 0.5):
        y_pred_score = self.predict_proba(x)
        return (y_pred_score > threshold).to(torch.int32)

In [9]:
torch.manual_seed(random_seed)
model = MLP(input_size=50, num_classes=NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
from coral_pytorch.dataset import levels_from_labelbatch
from coral_pytorch.losses import coral_loss


for epoch in range(num_epochs):

    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        ##### Convert class labels for CORAL
        levels = levels_from_labelbatch(class_labels + 1, num_classes=NUM_CLASSES)
        ###--------------------------------------------------------------------###

        features = features.to(DEVICE)
        levels = levels.to(DEVICE)
        logits, probas = model(features)

        #### CORAL loss
        loss = coral_loss(logits, levels)
        ###--------------------------------------------------------------------###

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        if not batch_idx % 200:
            print(
                "Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f"
                % (epoch + 1, num_epochs, batch_idx, len(train_loader), loss)
            )

Epoch: 001/100 | Batch 000/011 | Loss: 2.1775
Epoch: 002/100 | Batch 000/011 | Loss: 2.0583
Epoch: 003/100 | Batch 000/011 | Loss: 2.1252
Epoch: 004/100 | Batch 000/011 | Loss: 2.0616
Epoch: 005/100 | Batch 000/011 | Loss: 1.9817
Epoch: 006/100 | Batch 000/011 | Loss: 1.8627
Epoch: 007/100 | Batch 000/011 | Loss: 1.8738
Epoch: 008/100 | Batch 000/011 | Loss: 1.9805
Epoch: 009/100 | Batch 000/011 | Loss: 1.7715
Epoch: 010/100 | Batch 000/011 | Loss: 1.7385
Epoch: 011/100 | Batch 000/011 | Loss: 1.8067
Epoch: 012/100 | Batch 000/011 | Loss: 1.7782
Epoch: 013/100 | Batch 000/011 | Loss: 1.8222
Epoch: 014/100 | Batch 000/011 | Loss: 1.7732
Epoch: 015/100 | Batch 000/011 | Loss: 1.7711
Epoch: 016/100 | Batch 000/011 | Loss: 1.7409
Epoch: 017/100 | Batch 000/011 | Loss: 1.5250
Epoch: 018/100 | Batch 000/011 | Loss: 1.6339
Epoch: 019/100 | Batch 000/011 | Loss: 1.7606
Epoch: 020/100 | Batch 000/011 | Loss: 1.5905
Epoch: 021/100 | Batch 000/011 | Loss: 1.6928
Epoch: 022/100 | Batch 000/011 | L

In [14]:
from coral_pytorch.dataset import proba_to_label


def compute_mae_and_mse(model, data_loader, device):

    with torch.no_grad():

        mae, mse, acc, num_examples = 0.0, 0.0, 0.0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits, probas = model(features)
            predicted_labels = proba_to_label(probas).float()

            num_examples += targets.size(0)
            mae += torch.sum(torch.abs(predicted_labels - targets))
            mse += torch.sum((predicted_labels - targets) ** 2)

        mae = mae / num_examples
        mse = mse / num_examples
        return mae, mse

In [15]:
train_mae, train_mse = compute_mae_and_mse(model, train_loader, DEVICE)
test_mae, test_mse = compute_mae_and_mse(model, test_loader, DEVICE)

In [16]:
print(f"Mean absolute error (train/test): {train_mae:.2f} | {test_mae:.2f}")
print(f"Mean squared error (train/test): {train_mse:.2f} | {test_mse:.2f}")

Mean absolute error (train/test): 0.92 | 0.85
Mean squared error (train/test): 0.93 | 1.07


In [23]:
import torch


##########################
### SETTINGS
##########################

# Hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 128

# Architecture
NUM_CLASSES = 5

# Other
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Training on", DEVICE)

Training on cpu


In [21]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, num_classes, num_hidden_1=300, num_hidden_2=300):
        super().__init__()

        self.my_network = torch.nn.Sequential(
            nn.Linear(input_size, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            ### Specify CORN layer
            torch.nn.Linear(50, (num_classes - 1))
            ###--------------------------------------------------------------------###
        )

    def forward(self, x):
        logits = self.my_network(x)
        return logits


torch.manual_seed(random_seed)
model = MLP(input_size=50, num_classes=NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
from coral_pytorch.losses import corn_loss


for epoch in range(num_epochs):

    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        class_labels = class_labels.to(DEVICE)
        features = features.to(DEVICE)
        logits = model(features)

        #### CORN loss
        loss = corn_loss(logits, class_labels + 1, NUM_CLASSES)
        ###--------------------------------------------------------------------###

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        if not batch_idx % 200:
            print(
                "Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f"
                % (epoch + 1, num_epochs, batch_idx, len(train_loader), loss)
            )

Epoch: 001/100 | Batch 000/011 | Cost: 0.2115
Epoch: 002/100 | Batch 000/011 | Cost: 0.1162
Epoch: 003/100 | Batch 000/011 | Cost: 0.1362
Epoch: 004/100 | Batch 000/011 | Cost: 0.1631
Epoch: 005/100 | Batch 000/011 | Cost: 0.1534
Epoch: 006/100 | Batch 000/011 | Cost: 0.1528
Epoch: 007/100 | Batch 000/011 | Cost: 0.1057
Epoch: 008/100 | Batch 000/011 | Cost: 0.1110
Epoch: 009/100 | Batch 000/011 | Cost: 0.1342
Epoch: 010/100 | Batch 000/011 | Cost: 0.0992
Epoch: 011/100 | Batch 000/011 | Cost: 0.1270
Epoch: 012/100 | Batch 000/011 | Cost: 0.0764
Epoch: 013/100 | Batch 000/011 | Cost: 0.0928
Epoch: 014/100 | Batch 000/011 | Cost: 0.0637
Epoch: 015/100 | Batch 000/011 | Cost: 0.1006
Epoch: 016/100 | Batch 000/011 | Cost: 0.0764
Epoch: 017/100 | Batch 000/011 | Cost: 0.0971
Epoch: 018/100 | Batch 000/011 | Cost: 0.1042
Epoch: 019/100 | Batch 000/011 | Cost: 0.0603
Epoch: 020/100 | Batch 000/011 | Cost: 0.0797
Epoch: 021/100 | Batch 000/011 | Cost: 0.0916
Epoch: 022/100 | Batch 000/011 | C

In [27]:
from coral_pytorch.dataset import corn_label_from_logits


def compute_mae_and_mse(model, data_loader, device):

    with torch.no_grad():

        mae, mse, acc, num_examples = 0.0, 0.0, 0.0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            predicted_labels = corn_label_from_logits(logits).float()

            num_examples += targets.size(0)
            mae += torch.sum(torch.abs(predicted_labels - targets))
            mse += torch.sum((predicted_labels - targets) ** 2)

        mae = mae / num_examples
        mse = mse / num_examples
        return mae, mse

In [28]:
train_mae, train_mse = compute_mae_and_mse(model, train_loader, DEVICE)
test_mae, test_mse = compute_mae_and_mse(model, test_loader, DEVICE)
print(f"Mean absolute error (train/test): {train_mae:.2f} | {test_mae:.2f}")
print(f"Mean squared error (train/test): {train_mse:.2f} | {test_mse:.2f}")

Mean absolute error (train/test): 3.95 | 6.68
Mean squared error (train/test): 34.77 | 69.69
